In [1]:
# verify p-onset pickingg

import string
import argparse as ap
import sys
import os
import numpy as np

import obspy
import obspy.core as oc
from obspy import read_inventory

# from datetime import datetime, timedelta
import time

import pylab as plt
import pandas as pd
import h5py


import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import datetime
import warnings

mpl.rcParams['pdf.fonttype'] = 42

In [2]:
# directory PATH
METADIR = './ingv_metadata'
HDFDIR = './ingv_data'
FIGDIR='./ingv_output'

# file NAMES
# MetaData
events_metaname=os.path.join(METADIR, 'metadata_Instance_events_v2.csv') # Either Counts and GroundMotion
noise_metaname=os.path.join(METADIR, 'metadata_Instance_noise.csv')
# HDF5
events_hdfname=os.path.join(HDFDIR, 'Instance_events_counts.hdf5')
# gm_hdfname=os.path.join(HDFDIR,'Instance_events_gm_10k.hdf5')
noise_hdfname=os.path.join(HDFDIR,'Instance_noise.hdf5')
# Figures
# events_figname=os.path.join(FIGDIR,'waveforms_Evs.png')
# gm_figname=os.path.join(FIGDIR,'waveforms_Evs_gm.png')
# noise_figname=os.path.join(FIGDIR,'waveforms_Noise.png')

In [4]:
# read metadata for events and nois
events_metaData = pd.read_csv(
    events_metaname, keep_default_na=False,
    dtype={
        'station_location_code': object, 'source_mt_eval_mode': object,
        'source_mt_status': object,'source_mechanism_strike_dip_rake': object,
        'source_mechanism_moment_tensor': object, 'trace_p_arrival_time': object,
        'trace_s_arrival_time': object
    },
    low_memory=False)

noise_metaData = pd.read_csv(
    noise_metaname, dtype={'station_location_code': object},
    low_memory=False)

In [5]:
# read waveform
events_h5File = h5py.File(events_hdfname, 'r') # Events in counts
# gm_h5File = h5py.File(gm_hdfname, 'r')         # Events in ground motion units
noise_h5File = h5py.File(noise_hdfname, 'r')   # Noise in counts

In [9]:
from ingv_plot_waveform import split, build_stream, multiple_streams, multiple_streams_zoom

In [12]:
# figure labels
fignumbers = 'abcdefghijklmnopqrstuvwxyz'
alllabs = split(fignumbers)

In [11]:
events_metaData.info(verbose=True, null_counts=True)

/tmp/ipykernel_463/902701860.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  events_metaData.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159249 entries, 0 to 1159248
Data columns (total 115 columns):
 #    Column                            Non-Null Count    Dtype  
---   ------                            --------------    -----  
 0    source_id                         1159249 non-null  int64  
 1    station_network_code              1159249 non-null  object 
 2    station_code                      1159249 non-null  object 
 3    station_location_code             1159249 non-null  object 
 4    station_channels                  1159249 non-null  object 
 5    station_latitude_deg              1159249 non-null  float64
 6    station_longitude_deg             1159249 non-null  float64
 7    station_elevation_m               1159249 non-null  float64
 8    station_vs_30_mps                 1159249 non-null  float64
 9    station_vs_30_detail              1159249 non-null  object 
 10   source_origin_time                1159249 non-null  object 
 11   source_latitude_deg   

In [14]:
events_metaData.head()

,source_id,station_network_code,station_code,station_location_code,station_channels,station_latitude_deg,station_longitude_deg,station_elevation_m,station_vs_30_mps,station_vs_30_detail,...,trace_sa10_cmps2,trace_sa30_cmps2,trace_name,trace_GPD_P_number,trace_GPD_S_number,trace_EQT_number_detections,trace_EQT_P_number,trace_EQT_S_number,trace_deconvolved_units,source_type
0,10000541,MN,AQU,,HH,42.35400,13.40500,710.0,485.0,Vs30 extracted from ShakeMap,...,7.13432e-05,1.91573e-05,10000541.MN.AQU..HH,1.0,1.0,1.0,1.0,1.0,mps,earthquake
1,10000541,MN,AQU,,HL,42.35400,13.40500,710.0,485.0,Vs30 extracted from ShakeMap,...,0.0001351228,4.95397e-05,10000541.MN.AQU..HL,,,,,,mps2,earthquake
2,10000541,IV,ARRO,,EH,42.57917,12.76567,253.0,549.0,Vs30 extracted from ShakeMap,...,4.59196e-05,2.21168e-05,10000541.IV.ARRO..EH,2.0,1.0,1.0,1.0,1.0,mps,earthquake
3,10000541,IV,ASSB,,HH,43.04260,12.65870,734.0,412.0,Vs30 extracted from ShakeMap,...,6.82975e-05,1.47641e-05,10000541.IV.ASSB..HH,1.0,7.0,1.0,1.0,1.0,mps,earthquake
4,10000541,IV,ATCC,,EH,43.18514,12.63994,557.0,667.0,Vs30 extracted from ShakeMap,...,5.7025e-05,2.82292e-05,10000541.IV.ATCC..EH,1.0,3.0,1.0,1.0,1.0,mps,earthquake


In [13]:
# plot random traces centered to the p-onset time

lines = chosen_lines
# Kind of unit
units = len(lines)*["counts"]

# Do you want the traces filtered?
filt = False
freq_min=3.0
freq_max=20.0
#
dt = 1.0

nrow=int(len(lines)/plots_line)
ncol=plots_line

labs = alllabs[:(ncol*nrow)]

wftype='ev_c'
if wftype == 'ev_c':
    df = events_metaData
    h5 = events_h5File
elif wftype == 'ev_gm':
    df = events_metaData
    h5 = gm_h5File
elif wftype == 'noise':
    df = noise_metaData
    h5 = noise_h5File

multiple_streams_zoom(df,h5,lines,wftype,nrow,ncol,units,labs,filt,freq_min,freq_max,dt)
gm_figname=os.path.join(FIGDIR,'polarities_zoom_'+ str(random_state) + '.png')
# plt.savefig(gm_figname,dpi=300)

NameError: name 'chosen_lines' is not defined